# **Task-1:** Get the Parent Company Name

In [1]:
!pip install pandas openpyxl openai==1.34.0

In [2]:
# Read the data from the excel sheets
import pandas as pd

excel_data = pd.read_excel('data/Data for ML Test.xlsx', sheet_name=None)

examples_data_df  = excel_data["Example Sheet"]
to_do_df  = excel_data["To Do"]

In [4]:
examples_data_df.head()

,S.No,Suppler Name,Suppler Parent
0,1,IBM CORPORATION,IBM CORPORATION-
1,2,IBM CORP,IBM CORPORATION-
2,3,IBM CREDIT CORPORATION,IBM CORPORATION-
3,4,RED HAT INC,IBM CORPORATION-
4,5,RED HAT NC,IBM CORPORATION-


In [5]:
# Unique parent company
unique_parent_company = list(examples_data_df['Suppler Parent'].str.replace("-",'').unique())
unique_parent_company

['IBM CORPORATION',
 'DELL TECHNOLOGIES',
 'UNITED PARCEL SERVICE',
 'AON',
 'MICROSOFT',
 'UNITEDHEALTH GROUP',
 'WELLS FARGO FINANCIAL SERVICES']

In [7]:
to_do_df.head()

,S.No,Supplier Name,Supplier Parent
0,1,WELLCARE HEALTH PLANS,NaN
1,2,BANK OF AMERICA,NaN
2,3,MCKESSON,NaN
3,4,DIGITASLBI,NaN
4,5,OUTCOMESMTM,NaN


In [8]:
supplier_name_list:list = to_do_df["Supplier Name"].tolist()

In [ ]:
def update_unique_parent(parent_name):
    if not parent_name in unique_parent_company:
        unique_parent_company.append(parent_name)
    return 

In [9]:
def get_prompt(supplier_name: str):
    """Method to get the prompt"""
    # parent_list =','.join(unique_parent_company)
    return f"""Find out the parent company of the specified subsidiary company. 
The response should be a JSON object with the key name 'parent'.
---
examples:
user: RED HAT INC    
assistant: {{"parent":"IBM CORPORATION"}}

user: BOOMI INC        
assistant: {{"parent":"DELL TECHNOLOGIES"}}

user: {supplier_name}
assistant: """


In [10]:
import os
from getpass import getpass

azure_openai_endpoint = getpass("Enter Azure OpenAI Endpoint: ")
azure_openai_deployment = getpass("Enter Azure OpenAI Deployment Name: ")
azure_openai_key = getpass("Enter Azure OpenAI API Key: ")

os.environ["AZURE_OPENAI_ENDPOINT"] = azure_openai_endpoint
os.environ["DEPLOYMENT_NAME"] = azure_openai_deployment
os.environ["AZURE_OPENAI_KEY"] = azure_openai_key

In [11]:
import os
from openai import AzureOpenAI

azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
deployment_name = os.environ["DEPLOYMENT_NAME"]
api_key = os.environ["AZURE_OPENAI_KEY"]
api_version = "2024-02-01"

def llm(prompt:str):
    """Azure OpenAI API call"""

    client = AzureOpenAI(
            azure_endpoint=azure_endpoint,
            api_key=api_key,
            api_version=api_version
        )
    completion = client.chat.completions.create(
        model=deployment_name,  # model = "deployment_name"
        messages= [
        {"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=800,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    return completion.choices[0].message.content

In [12]:
import json

def get_parent_company(*supplier_names):
    parent_company:list = []
    for each_supplier in supplier_names:
        out = llm(get_prompt(each_supplier))
        try:
            json_data = json.loads(out)
            json_data["raw"] = out
            update_unique_parent(json_data['parent'])
        except Exception as e:
            json_data =  {"raw": out,"parent": " "}
        parent_company.append(json_data)
        # print(f"S: {each_supplier} :: P: {json_data}")
    return parent_company

In [13]:
out_parent_company_list = get_parent_company(*supplier_name_list)

In [28]:
to_do_df['raw'] = None
for i in range(len(out_parent_company_list)):
    parent = out_parent_company_list[i]['parent']
    if parent:
        to_do_df.at[i,'Supplier Parent'] = parent
    else:
        to_do_df.at[i,'raw'] = out_parent_company_list[i]['raw']

In [29]:
to_do_df.to_excel('data/output_after_llm_to_do.xlsx', index=False)  

In [41]:
import pandas as pd

df = pd.read_excel('data/output_after_llm_to_do.xlsx')


In [43]:
df.head()

,S.No,Supplier Name,Supplier Parent,raw
0,1,WELLCARE HEALTH PLANS,CENTENE CORPORATION,NaN
1,2,BANK OF AMERICA,BANK OF AMERICA CORPORATION,NaN
2,3,MCKESSON,MCKESSON CORPORATION,NaN
3,4,DIGITASLBI,PUBLICIS GROUPE,NaN
4,5,OUTCOMESMTM,COTIVITI HOLDINGS INC,NaN


In [47]:
df

,S.No,Supplier Name,Supplier Parent,raw
0,1,WELLCARE HEALTH PLANS,CENTENE CORPORATION,NaN
1,2,BANK OF AMERICA,BANK OF AMERICA CORPORATION,NaN
2,3,MCKESSON,MCKESSON CORPORATION,NaN
3,4,DIGITASLBI,PUBLICIS GROUPE,NaN
4,5,OUTCOMESMTM,COTIVITI HOLDINGS INC,NaN
...,...,...,...,...
1529,1530,KIMCO HORSHAM LP SPAH1337A-LCVS///0,KIMCO REALTY CORPORATION,NaN
1530,1531,PNC BANK 1008175985 HERMAN LIPSITZ,PNC FINANCIAL SERVICES GROUP,NaN
1531,1532,TAYLOR CORPORATION,TAYLOR CORPORATION,NaN
1532,1533,LEXISNEXIS RISK SOLUTIONS INC,RELX GROUP,NaN


# **Task-2:** Clustering or classification 

In [55]:
# Load the data from the excel
import pandas as pd
import numpy as np

excel_data = pd.read_excel('data/Data for ML Test.xlsx', sheet_name=None)
examples_data_df  = excel_data["Example Sheet"]

output_from_llm_sheet = pd.read_excel('data/output_after_llm_to_do.xlsx')[['Supplier Name','Supplier Parent']]

In [46]:
# Check the NaN in the sheet
print(output_from_llm_sheet.isna().any())
print(examples_data_df.isna().any())

output_from_llm_sheet[output_from_llm_sheet.isna().any(axis=1)]

Supplier Name      False
Supplier Parent     True
dtype: bool
S.No              False
Suppler Name      False
Suppler Parent    False
dtype: bool


,Supplier Name,Supplier Parent
189,THE REGENTS OF THE UNIVERSITY OF CALIFORNIA,NaN
621,CREEHAN & COMPANY,NaN


In [47]:
# drop the row having any NaN value
index_to_drop = output_from_llm_sheet[output_from_llm_sheet['Supplier Parent'].isna()].index
output_from_llm_sheet.drop(index_to_drop,inplace=True)

In [48]:
# x_ and y_ are inputs and outputs for the classfication model
x_ = np.append(examples_data_df['Suppler Name'].to_numpy(),output_from_llm_sheet['Supplier Name'].to_numpy())
y_ = np.append(examples_data_df['Suppler Parent'].to_numpy(),output_from_llm_sheet['Supplier Parent'].to_numpy())
x_.size, y_.size

(1634, 1634)

In [49]:
x_,y_

(array(['IBM CORPORATION', 'IBM CORP', 'IBM CREDIT CORPORATION', ...,
        'TAYLOR CORPORATION', 'LEXISNEXIS RISK SOLUTIONS INC',
        "SSM ST MARY'S HOSPITAL"], dtype=object),
 array(['IBM CORPORATION-', 'IBM CORPORATION-', 'IBM CORPORATION-', ...,
        'TAYLOR CORPORATION', 'RELX GROUP', 'SSM HEALTH CARE CORPORATION'],
       dtype=object))

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [51]:
# words to numerical vector
vectorizer = TfidfVectorizer()
x_vectors = vectorizer.fit_transform(x_)

x_train,x_test,y_train,y_test = train_test_split(x_vectors,y_,test_size=0.2,shuffle=True)

In [52]:
# Using Random Forest Clasifier from sklearn
classifier = RandomForestClassifier()
classifier.fit(x_train, y_train)

RandomForestClassifier()

In [53]:
# Test the trained model
y_pred = classifier.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5229357798165137
Classification Report:
                                                         precision    recall  f1-score   support

                           8032 LIBERTY ROAD OWNER LLC       0.00      0.00      0.00         1
                                    ABM INDUSTRIES INC       1.00      1.00      1.00         1
                                             ACCENTURE       0.00      0.00      0.00         0
                               ADVENTIST HEALTH SYSTEM       0.00      0.00      0.00         1
                            ALBERTSON'S COMPANIES, LLC       0.00      0.00      0.00         1
                                  ALBERTSONS COMPANIES       1.00      0.33      0.50         3
                             ALBERTSONS COMPANIES, LLC       0.00      0.00      0.00         0
                    AMERICAN REALTY CAPITAL PROPERTIES       0.00      0.00      0.00         0
              AMERICAN REALTY CAPITAL PROPERTIES, INC.       0.00      0.00      0

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/codespace/.loca